In [1]:
import pickle
import os
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
if 'df' in globals(): del df

for folder in os.listdir('/mnt/G-SSD/marco_mestrado/master_debias/UI_label_images/labeled_images/'):
    for file in os.listdir('/mnt/G-SSD/marco_mestrado/master_debias/UI_label_images/labeled_images/'+folder):
        with open('/mnt/G-SSD/marco_mestrado/master_debias/UI_label_images/labeled_images/'+folder+'/'+file, 'rb') as f:
            imgs_list = pickle.load(f)
        
        if 'df' not in globals():
            df = pd.DataFrame(imgs_list)
        else:
            df = pd.concat([df, pd.DataFrame(imgs_list)], axis=0, ignore_index=True)
df.drop('index', axis=1, inplace=True)
df['orig_deb'] = df['img_path'].str.contains('original').replace(True,'original').replace(False,'debiased')
df.shape

(2400, 9)

In [3]:
# Image Annotation by User
df.groupby('user_name').count()['img_path']

user_name
lucas    1200
marco    1200
Name: img_path, dtype: int64

In [4]:
# Unique Annotations
users = np.sort(df['user_name'].unique())
for user in users:
    print(user+'   \t'+str(len(df.loc[df['user_name']==user]['img_path'].unique())))

lucas   	1200
marco   	1200


In [14]:
import os
import pickle
import pandas as pd

# Define the folder path and user
user = 'marco/'
base_path = f'/mnt/G-SSD/marco_mestrado/master_debias/UI_label_images/labeled_images/{user}'

# List to hold the data
data = []

# Process files in the folder
for file in os.listdir(base_path):
    file_path = os.path.join(base_path, file)
    
    # Load the pickle content
    with open(file_path, 'rb') as f:
        content = pickle.load(f)
    
    # Extract img_path from index 0
    if content and isinstance(content, list) and 'img_path' in content[0]:
        img_path = content[0]['img_path']
        data.append({'pickle_file': file, 'img_path': img_path})
    else:
        data.append({'pickle_file': file, 'img_path': 'N/A'})

# Create and display the DataFrame
df = pd.DataFrame(data)

duplicates = df[df.duplicated(subset='img_path', keep=False)]

# Display the duplicate files
print(duplicates)


                               pickle_file  \
1   annotated_imgs__1735944028.7630224.pkl   
33  annotated_imgs__1735930908.3873463.pkl   

                                             img_path  
1   /static/images/firefighter/firefighter_debiase...  
33  /static/images/firefighter/firefighter_debiase...  


In [26]:
# Print Stats for Protected Attributes
for prompt in ['the face of firefighter','the face of a nurse','the face of a business leader']:
    print(f'\n\033[1m Prompt: \033[93m{prompt}\033[0m')
    for protected_att in ['gender','race','apparent_age']:
        stats = pd.DataFrame() 
        for orig_deb in ['original','debiased']:
            stats_new = df[(df['prompt']==prompt) & (df['orig_deb']==orig_deb)].groupby(protected_att).count()['img_path'].to_frame()
            stats = pd.concat([stats, stats_new], axis=1, ignore_index=True)
        pct = stats / stats.sum() * 100
        display(pct.rename(columns={0:'original', 1:'debiased'}))
    print(f'Samples from Original Model: {stats.sum()[0]} | Samples from Debiased Model: {stats.sum()[1]}')


 Prompt: the face of firefighter


,original,debiased
gender,,
female,1.22549,0.493827
male,98.77451,99.506173


,original,debiased
race,,
black,9.558824,46.913580
indian,1.715686,40.987654
white,88.725490,11.358025
asian,NaN,0.740741


,original,debiased
apparent_age,,
midAge,31.127451,72.839506
young,68.872549,26.419753
elderly,NaN,0.740741


Samples from Original Model: 408.0 | Samples from Debiased Model: 405.0

 Prompt: the face of a nurse


...

Samples from Original Model: 403 | Samples from Debiased Model: 407

 Prompt: the face of a business leader


,original,debiased
gender,,


,original,debiased
race,,


,original,debiased
apparent_age,,


Samples from Original Model: 0.0 | Samples from Debiased Model: 0.0


In [5]:
# EXPORT TO CSV INTO LOCAL MACHINE
df.to_csv('debias_experiment_export_test.csv')
# On Windows cmd: scp jordankp@motox:/home/jordankp/sdxl/UI_label_images/debias_experiment_export.csv .